# Statistics about SQL

In [4]:
import sqlite3
import pandas as pd

In [11]:
con = sqlite3.connect("../aisrt.db")
df_raw_history = pd.read_sql_query("SELECT * from HistoricPosition", con)
print(len(df_raw_history))
df_raw_history

1443952


,id,mmsi,lat,long,timestamp
0,1,257048880,59.123343,5.068393,2022-08-29 20:37:51.573845
1,2,259577000,60.814000,5.016903,2022-08-29 20:37:51.591357
2,3,257221400,59.817168,5.269052,2022-08-29 20:37:51.639492
3,4,257096170,63.950278,9.644062,2022-08-29 20:37:52.062206
4,5,257458000,63.120615,7.656317,2022-08-29 20:37:52.097901
...,...,...,...,...,...
1443947,1443948,311000116,59.762850,5.497688,2022-09-02 19:31:20.055783
1443948,1443949,257912700,59.207405,10.894040,2022-09-02 19:31:20.102444
1443949,1443950,257704000,62.477928,6.225527,2022-09-02 19:31:20.542807
1443950,1443951,257265000,59.902972,10.739090,2022-09-02 19:31:20.581182


In [13]:
df_raw_history.dtypes

id             int64
mmsi           int64
lat          float64
long         float64
timestamp     object
dtype: object

In [26]:
(df_raw_history
  .assign(time=pd.to_datetime(df_raw_history["timestamp"]))
  .drop(columns=["id", "timestamp"])
  .astype({"mmsi" : "category"})
).to_parquet("history_small.parquet.gz", compression="gzip")

In [12]:
df_raw_imo_vessels = pd.read_sql("SELECT * from ImoVesselCodes", con)
print(len(df_raw_imo_vessels))
df_raw_imo_vessels

3524


,mmsi,imo,name,flag,type
0,259961000,9591856,NORMAND SERVER,,70
1,257671900,0,RESTLESS CHALLENGER,,31
2,257101740,0,STROMOYGUTT,,30
3,257000260,0,FROYSPRINT,,33
4,210803000,9374727,FRAKT SUND,,70
...,...,...,...,...,...
3519,538006753,9200419,FEDERAL ASAHI,,70
3520,229013000,9498274,RODOPI,,70
3521,235108128,9709922,GRAMPIAN DYNASTY,,51
3522,477345600,9834325,LNG MEGREZ,,81
